In [ ]:
import pandas as pd


#load processed and resampled train dataset
df = pd.read_csv('processed/train_dataset.csv')

In [ ]:
df_treated = df[df['ind_recommended']==1]
df_control = df[df['ind_recommended']==0]

In [ ]:
treated_minority_len = len(df_treated[df_treated['activation'] == 1])
control_minority_len = len(df_control[df_control['activation'] == 1])

6946
42291


In [ ]:
from sklearn.utils import resample

treated_minority = resample(df_treated[df_treated['activation'] == 1],replace=True, n_samples=treated_minority_len, random_state=42)
treated_majority = resample(df_treated[df_treated['activation'] == 0],replace=True, n_samples=treated_minority_len, random_state=42)

control_minority = resample(df_control[df_control['activation'] == 1],replace=True, n_samples=control_minority_len, random_state=42)
control_majority = resample(df_control[df_control['activation'] == 0],replace=True, n_samples=control_minority_len, random_state=42)

df_treated = pd.concat([treated_minority, treated_majority])
df_control = pd.concat([control_minority, control_majority])

In [ ]:
df_treated = process(df_treated)
y_treated = df_treated['activation']
X_treated = df_treated.drop(['merchant','customer','activation','ind_recommended'], axis=1)

In [ ]:
df_control = process(df_control)
y_control = df_control['activation']
X_control = df_control.drop(['merchant','customer','activation','ind_recommended'], axis=1)

In [ ]:
from xgboost import XGBRegressor

s1_treated = XGBRegressor()
s1_treated.fit(X_treated, y_treated)
s1_treated_result = s1_treated.predict(X_control)

s1_treated.save_model('s1_treated.json')

In [ ]:
s1_control = XGBRegressor()
s1_control.fit(X_control, y_control)
s1_control_result = s1_control.predict(X_treated)

s1_control.save_model('s1_control.json')

In [ ]:
treated_imputed_ite = (y_treated - s1_control_result)
control_imputed_ite = (s1_treated_result - y_control)

In [ ]:
s2_treated = XGBRegressor()
s2_treated.fit(X_treated, treated_imputed_ite)
s2_treated_result = s2_treated.predict(X_treated)

s2_treated.save_model('s2_treated.json')

In [ ]:
s2_control = XGBRegressor()
s2_control.fit(X_control, control_imputed_ite)
s2_control_result = s2_control.predict(X_control)

s2_control.save_model('s2_control.json')